In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zainabhaddad/reviews-cell-phones-and-accessories-5")

print("Path to dataset files:", path)

Path to dataset files: /Users/sagarguttal/.cache/kagglehub/datasets/zainabhaddad/reviews-cell-phones-and-accessories-5/versions/1


In [2]:
import gensim
import pandas as pd

In [3]:
df = pd.read_json(r"/Users/sagarguttal/Work/Natural-Language-Processing/Datasets/Cell_Phones_and_Accessories_5.json", lines=True)
df.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [4]:
df.shape

(194439, 9)

### Preprocessing and Tokenization

In [5]:
df['reviewText'][0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [6]:
gensim.utils.simple_preprocess(df['reviewText'][0])

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [7]:
review_text = df['reviewText'].apply(gensim.utils.simple_preprocess)
review_text.head()

0    [they, look, good, and, stick, good, just, don...
1    [these, stickers, work, like, the, review, say...
2    [these, are, awesome, and, make, my, phone, lo...
3    [item, arrived, in, great, time, and, was, in,...
4    [awesome, stays, on, and, looks, great, can, b...
Name: reviewText, dtype: object

### Train word2vec model

In [8]:
model = gensim.models.Word2Vec(window=10, vector_size=100, min_count=2, epochs=7,sg=1) # sg parameter is skipgram = 1 means skipgram model architectire True
model

## Build the vocabulary

In [9]:
model.build_vocab(review_text)

In [10]:
model.epochs, model.window, model.corpus_count

(7, 10, 194439)

In [11]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(86107121, 117416565)

### Save the model

In [12]:
model.save("trained_word2vec_model.model")

### FInd the similar word and similarity between the words

In [13]:
model.wv.most_similar("bad", topn=3)

[('terrible', 0.7115426063537598),
 ('horrible', 0.6776736378669739),
 ('ok', 0.6636120676994324)]

In [14]:
model.wv.similarity(w1="cheap",w2="inexpensive")

0.6581166

In [15]:
matrix = model.wv.vectors ## extract the word embeddings matrix
matrix.shape ## unique words, num feature extracted

(35561, 100)

## Forword pass example how model input one hot codeed vectors passed to model and predict the nearest words

In [17]:
W_out = model.syn1neg
W_out.shape ## Output weights from model after applying the softmax


(35561, 100)

In [24]:
vocab_lists = model.wv.index_to_key
print("vacabulary list upto 10 words", vocab_lists[:10])
print("total Len of vacabulary list", len(vocab_lists))

vacabulary list upto 10 words ['the', 'it', 'and', 'to', 'is', 'this', 'of', 'for', 'my', 'that']
total Len of vacabulary list 35561


In [32]:
word = "sea"
idx = model.wv.key_to_index[word]
vec = model.wv.vectors[idx]
count = model.wv.get_vecattr(word, "count")
idx, vec, count

(5931,
 array([-0.00712258,  0.2359466 , -0.03114446, -0.22443694,  0.25148937,
        -0.20807847, -0.26968727,  0.22847642, -0.05672771, -0.02954084,
         0.23332837, -0.09763193,  0.4943103 ,  0.16879876,  0.06176304,
         0.2478795 , -0.56947017,  0.25061676, -0.08822034, -0.18338726,
         0.12661164,  0.34964955,  0.17069395, -0.19793949, -0.30371   ,
         0.49247393,  0.22642732, -0.21975815,  0.18219912,  0.15755494,
         0.13632259,  0.38600272,  0.03332593,  0.03172928, -0.11848735,
         0.26687402, -0.19381797,  0.05069618, -0.07853325,  0.14375773,
         0.17922062,  0.39787996, -0.39109248,  0.388078  , -0.47051585,
         0.10893855,  0.4414546 , -0.43341818, -0.1433949 ,  0.7340807 ,
         0.5267529 ,  0.6316635 , -0.27696586,  0.347123  , -0.19481823,
        -0.27216196,  0.21183   ,  0.28836504, -0.57246274,  0.53933406,
         0.10666623, -0.08862111, -0.10123557,  0.2271278 , -0.11939315,
         0.21780343, -0.17280369, -0.4413308

In [45]:
import numpy as np
one_hot = np.zeros(shape=(1, len(vocab_lists)))
one_hot.shape

(1, 35561)

In [46]:
word = "sea"
one_hot[0, model.wv.key_to_index[word]] = 1

In [49]:
one_hot[0, model.wv.key_to_index[word]]

1.0

In [57]:
matrix.shape, W_out.shape, W_out.T.shape

((35561, 100), (35561, 100), (100, 35561))

In [53]:
a_proj = np.dot(one_hot, matrix) ## EMbedding of the word sea
a_proj.shape

(1, 100)

In [54]:
a_proj 

array([[-0.00712258,  0.2359466 , -0.03114446, -0.22443694,  0.25148937,
        -0.20807847, -0.26968727,  0.22847642, -0.05672771, -0.02954084,
         0.23332837, -0.09763193,  0.49431029,  0.16879876,  0.06176304,
         0.24787951, -0.56947017,  0.25061676, -0.08822034, -0.18338726,
         0.12661164,  0.34964955,  0.17069395, -0.19793949, -0.30371001,
         0.49247393,  0.22642732, -0.21975815,  0.18219912,  0.15755494,
         0.13632259,  0.38600272,  0.03332593,  0.03172928, -0.11848735,
         0.26687402, -0.19381797,  0.05069618, -0.07853325,  0.14375773,
         0.17922062,  0.39787996, -0.39109248,  0.388078  , -0.47051585,
         0.10893855,  0.44145459, -0.43341818, -0.1433949 ,  0.73408067,
         0.52675289,  0.6316635 , -0.27696586,  0.347123  , -0.19481823,
        -0.27216196,  0.21183001,  0.28836504, -0.57246274,  0.53933406,
         0.10666623, -0.08862111, -0.10123557,  0.22712781, -0.11939315,
         0.21780343, -0.17280369, -0.44133082,  0.1

In [58]:
z_out = np.dot(a_proj, W_out.T)
z_out.shape

(1, 35561)

In [59]:
a_out = np.exp(z_out)/np.sum(np.exp(z_out))
a_out.shape

(1, 35561)

In [64]:
a_out = a_out.flatten()
indices = np.argsort(a_out) ## Sort in accedensing order
indices = indices[::-1] ## Sort in descending order

for i in range(10): ## TOp 10 nearest words word sea
    print(f"index number {indices[i]} and corrsponding word {model.wv.index_to_key[indices[i]]} and score {a_out[indices[i]]}" )


index number 5931 and corrsponding word sea and score 0.06034282671022149
index number 7159 and corrsponding word tan and score 0.0015840976364057248
index number 8618 and corrsponding word jet and score 0.0010346074505529377
index number 7451 and corrsponding word ocean and score 0.0009873535977361304
index number 7215 and corrsponding word shops and score 0.0009802432025984933
index number 8443 and corrsponding word garden and score 0.0009225551221556965
index number 3381 and corrsponding word el and score 0.0008612473022258687
index number 10610 and corrsponding word blues and score 0.0008481035638122385
index number 3769 and corrsponding word la and score 0.0008480412453309058
index number 4713 and corrsponding word eacute and score 0.0008022477086344492
